In [215]:
import os
import importlib
import numpy as np
import pandas as pd
import seaborn as sns
import pylab as lp

In [216]:
import analyze2p.receptive_fields.utils as rfutils
import analyze2p.objects.sim_utils as su

import analyze2p.plotting as pplot
import analyze2p.aggregate_datasets as aggr
import analyze2p.utils as hutils


In [217]:
#### Plotting params
visual_areas, area_colors = pplot.set_threecolor_palette()
pplot.set_plot_params(labelsize=6, lw_axes=0.25)
bw_colors = dict((v, [0.7]*3) for v in visual_areas)

In [218]:
importlib.reload(aggr)

<module 'analyze2p.aggregate_datasets' from '/home/julianarhee/Repositories/rat-2p-area-characterizations/analyze2p/aggregate_datasets.py'>

In [219]:
#### Set trace ID and FOV/state type
aggregate_dir = '/n/coxfs01/julianarhee/aggregate-visual-areas'
rootdir='/n/coxfs01/2p-data'
fov_type = 'zoom2p0x'
state = 'awake'
# ------------------------------------------------------------------
# Load all metdata and assigned cells
sdata, cells0, missing_ = aggr.get_aggregate_info(visual_areas=visual_areas, 
                                        return_missing=True,
                                        create_new=True, return_cells=True)

experiment_list = ['rfs10', 'rfs'] 
meta = sdata[sdata.experiment.isin(experiment_list)].copy()

Loading old...


OSError: [Errno 112] Host is down: '/n/coxfs01/julianarhee/aggregate-visual-areas/dataset_info.pkl'

In [182]:
import analyze2p.retinotopy.utils as retutils

In [198]:
dk = '20190508_JC083_fov1'
all_retinos = retutils.get_average_mag_across_pixels(dk)
retinorun = all_retinos.iloc[all_retinos[1].idxmax()][0]
print(retinorun)


retino_run1


#### aggr.get_cells_by_area()

In [185]:
excluded_datasets = ['20190602_JC080_fov1', '20190605_JC090_fov1',
                     '20191003_JC111_fov1', 
                     '20191104_JC117_fov1', '20191104_JC117_fov2', 
                     #'20191105_JC117_fov1',
                     '20191108_JC113_fov1', '20191004_JC110_fov3',
                     '20191008_JC091_fov'] 
missing_segmentation=[]
d_ = []
for (animalid, session, fov, datakey), g in sdata.groupby(['animalid', 'session', 'fov', 'datakey']):
    if datakey in excluded_datasets:
        continue
    roi_assignments=dict()
    try:
        # Get best retino
        all_retinos = retutils.get_average_mag_across_pixels(datakey)     
        retinorun = all_retinos.iloc[all_retinos[1].idxmax()][0]
        #retinorun = all_retinos.loc[all_retinos[1].idxmax()][0] 
        rois_ = roiutils.load_roi_assignments(animalid, session, \
                                            fov, retinorun=retinorun)
        for varea, rlist in rois_.items():
            if varea not in roi_assignments.keys():
                roi_assignments[varea] = []
            roi_assignments[varea].extend(rlist)
    except Exception as e:
        if verbose:
            print("... skipping %s (%s)" % (datakey, retinorun))
        missing_segmentation.append((datakey, retinorun))
        continue 
    for varea, rlist in roi_assignments.items():
        if hutils.isnumber(varea):
            continue  
        tmpd = pd.DataFrame({'cell': list(set(rlist))})
        session, animalid, fovn = hutils.split_datakey_str(datakey)
        metainfo = {'visual_area': varea, 
                    'animalid': animalid, 'session': session,
                    'fov': fov, 'fovnum': int(fovn[3:]), 
                    'datakey': datakey}
        tmpd = hutils.add_meta_to_df(tmpd, metainfo)
        d_.append(tmpd)

cells = pd.concat(d_, axis=0).reset_index(drop=True)
cells = cells[~cells['datakey'].isin(excluded_datasets)]



AttributeError: 'Series' object has no attribute 'idmax'

In [197]:
all_retinos.iloc[all_retinos[1].idxmax()][0]



'retino_run1'

In [186]:
all_retinos.idmax()

AttributeError: 'DataFrame' object has no attribute 'idmax'

In [121]:
importlib.reload(rfutils)

<module 'analyze2p.receptive_fields.utils' from '/home/julianarhee/Repositories/rat-2p-area-characterizations/analyze2p/receptive_fields/utils.py'>

In [161]:
do_spherical_correction=False
response_type='dff'

rf_fit_desc = rfutils.get_fit_desc(response_type=response_type, 
                                 do_spherical_correction=do_spherical_correction)
# ALL rfdata, including rf-5 and rf-10
rfdata, no_fit, no_eval = rfutils.aggregate_rfdata(meta, cells0, fit_desc=rf_fit_desc,
                                 reliable_only=False, return_missing=True)

N dpaths: 62, N unfit: 0
N datasets included: 62, N sessions excluded: 4


In [162]:
rfdata.shape

(2454, 27)

In [163]:
rfdata[['visual_area', 'datakey', 'experiment']].drop_duplicates()\
    .groupby(['visual_area', 'datakey']).count().max()

experiment    2
dtype: int64

In [164]:
# Combine all rfdata to be AVERAGE or rf-5 or rf-10, WHATEVER we have
# Most liberal method
rfdf = rfutils.average_rfs(rfdata, keep_experiment=False)


In [165]:
rfdf[['visual_area', 'datakey', 'experiment']].drop_duplicates()\
    .groupby(['visual_area', 'datakey']).count().max()

experiment    1
dtype: int64

In [166]:
len(rfdf[rfdf.datakey=='20190508_JC083_fov1']['cell'].unique())


80

In [158]:
rfdf[rfdf.datakey=='20190508_JC083_fov1'].shape

(92, 27)

In [168]:
bad_assignment = []
for (dk, ri), g in rfdf.groupby(['datakey', 'cell']):
    if g.shape[0]>1:
        bad_assignment.append((dk, ri))
        
bad_assignment       

[('20190508_JC083_fov1', 197.0),
 ('20190508_JC083_fov1', 204.0),
 ('20190508_JC083_fov1', 209.0),
 ('20190508_JC083_fov1', 210.0),
 ('20190508_JC083_fov1', 211.0),
 ('20190508_JC083_fov1', 215.0),
 ('20190508_JC083_fov1', 220.0),
 ('20190508_JC083_fov1', 221.0),
 ('20190508_JC083_fov1', 224.0),
 ('20190508_JC083_fov1', 226.0),
 ('20190508_JC083_fov1', 227.0),
 ('20190508_JC083_fov1', 232.0)]

In [160]:
 rfdf[(rfdf.datakey=='20190508_JC083_fov1') & (rfdf['cell']==204)]

,datakey,cell,x0,y0,fwhm_x,fwhm_y,theta,offset,amplitude,r2,...,ratio_xy,theta_c,theta_Mm_deg,major_axis,minor_axis,anisotropy,theta_Mm_c,aniso_index,visual_area,experiment
756,20190508_JC083_fov1,204.0,60.771195,1.988298,50.164217,19.823805,0.180837,0.002832,-0.079042,0.778632,...,2.530504,0.180837,10.361222,21.346475,8.435661,0.433509,0.180837,-0.277573,Lm,average_rfs
1136,20190508_JC083_fov1,204.0,60.771195,1.988298,50.164217,19.823805,0.180837,0.002832,-0.079042,0.778632,...,2.530504,0.180837,10.361222,21.346475,8.435661,0.433509,0.180837,-0.277573,V1,average_rfs


In [169]:
curr_rois = [r[1] for r in bad_assignment]
rfdata[(rfdata.datakey=='20190508_JC083_fov1') & (rfdata['cell'].isin(curr_rois))]

       

,x0,y0,fwhm_x,fwhm_y,theta,offset,amplitude,r2,cell,datakey,...,fx,fy,ratio_xy,theta_c,theta_Mm_deg,major_axis,minor_axis,anisotropy,theta_Mm_c,aniso_index
804,53.957145,2.548797,27.746813,13.209007,0.060078,-0.005116,0.312276,0.898802,197.0,20190508_JC083_fov1,...,11.785853,0.708918,2.100598,0.060078,3.442190,11.807154,5.620854,0.354963,0.060078,-0.312338
805,60.771195,1.988298,50.164217,19.823805,0.180837,0.002832,-0.079042,0.778632,204.0,20190508_JC083_fov1,...,20.998387,3.839237,2.530504,0.180837,10.361222,21.346475,8.435661,0.433509,0.180837,-0.277573
806,27.697528,-5.058667,28.890404,17.639930,5.366161,-0.005104,0.177550,0.703980,209.0,20190508_JC083_fov1,...,7.476900,9.758751,1.637784,-0.917024,307.458392,12.293789,7.506353,0.241788,-0.917024,0.142072
807,48.238708,1.042879,17.429473,37.888430,4.995223,-0.009401,0.197339,0.865207,210.0,20190508_JC083_fov1,...,4.499508,15.482153,0.460021,-1.287962,16.205210,16.122736,7.416797,0.369843,0.282834,-0.163413
808,63.632065,2.016490,52.431512,18.421903,0.239581,-0.005698,0.216859,0.909220,211.0,20190508_JC083_fov1,...,21.674016,5.294367,2.846151,0.239581,13.726974,22.311282,7.839108,0.480000,0.239581,-0.252196
809,60.533325,10.287261,22.124937,55.446373,5.260519,-0.001223,0.074922,0.523204,215.0,20190508_JC083_fov1,...,12.294753,20.137661,0.399033,-1.022666,31.405540,23.594201,9.414867,0.429559,0.548130,0.018121
810,55.281476,-4.843543,30.948800,16.409598,6.252317,-0.001748,0.057979,0.692741,220.0,20190508_JC083_fov1,...,13.163428,0.406457,1.886018,-0.030868,358.231397,13.169702,6.982808,0.307004,-0.030868,-0.288054
811,47.628801,-3.631355,36.595432,19.639671,6.096561,-0.002028,0.029643,0.673346,221.0,20190508_JC083_fov1,...,15.302126,2.889369,1.863342,-0.186624,349.307224,15.572524,8.357307,0.301516,-0.186624,-0.189628
812,50.073437,-3.621598,18.096031,13.073935,5.832349,-0.001780,0.116033,0.725573,224.0,20190508_JC083_fov1,...,6.931035,3.355222,1.384130,-0.450836,334.168994,7.700439,5.563377,0.161120,-0.450836,-0.020714
813,52.580983,3.081586,44.776908,20.032769,0.356554,0.002214,-0.040034,0.762499,226.0,20190508_JC083_fov1,...,17.855606,6.650744,2.235183,0.356554,20.429039,19.054003,8.524582,0.381797,0.356554,-0.115267


In [174]:
c1 = cells0[(cells0.datakey=='20190508_JC083_fov1') & (cells0.visual_area=='V1')]['cell']
c2 = cells0[(cells0.datakey=='20190508_JC083_fov1') & (cells0.visual_area=='Lm')]['cell']

In [178]:
len(np.intersect1d(c1, c2))


42

In [128]:
dst_dir = os.path.join(aggregate_dir, 'receptive-fields', 'dataframes')
if not os.path.exists(dst_dir):
    os.makedirs(dst_dir)
print(dst_dir)

poly_fpath = os.path.join(dst_dir, 'average_polys.pkl')
if os.path.exists(poly_fpath):
    with open(poly_fpath, 'rb') as f:
        POLYS = pkl.load(f)
else:
    POLYS = None

/n/coxfs01/julianarhee/aggregate-visual-areas/receptive-fields/dataframes


In [130]:
# Process all cells in fov
cols = [c for c in rfdf.columns if c!='visual_area']
by_dkey = rfdf[cols].drop_duplicates()
# Add new, if needed
check_these={}
add_list=[]
for dk, curr_rfs in by_dkey.groupby('datakey'):
    if POLYS is not None:
        found_ = POLYS[(POLYS.datakey==dk)]
        need_ = curr_rfs[curr_rfs['cell'].isin(found_['cell'].values)]
    else:
        need_ = curr_rfs.copy()
    rf_polys, check_rfs = su.get_rf_polys(need_, check_invalid=True)
    if len(check_rfs)>0:
        check_these[dk]= check_rfs
    rf_polys['datakey'] = dk
    add_list.append(rf_polys)

#     rfs_ = params_deg_to_pixels(curr_rfs)
#     rf_screen = rf_to_screen(ri, rfs_, resolution=resolution)
#     rpoly = image_to_poly(rf_screen.astype(np.uint8))
    

In [132]:
len(rfdf['datakey'].unique())

46

In [136]:
rf_polys

,poly,cell,datakey
0,"POLYGON ((964 267, 963 268, 950 268, 949 269, ...",66.0,20191111_JC120_fov1
1,"POLYGON ((1306 210, 1305 211, 1296 211, 1295 2...",75.0,20191111_JC120_fov1
2,"POLYGON ((162 526, 161 527, 153 527, 152 528, ...",84.0,20191111_JC120_fov1
3,"POLYGON ((1122 405, 1121 406, 1099 406, 1098 4...",183.0,20191111_JC120_fov1


In [29]:
rfs_ = su.params_deg_to_pixels(curr_rfs)
for ri, r_ in rfs_.groupby('cell'):
    rf_screen = su.rf_to_screen(ri, r_)
    rpoly = su.image_to_poly(rf_screen.astype(np.uint8))

ValueError: invalid literal for int() with base 10: 'x0_pix'

In [38]:
rf_polys

,poly,cell
0,"POLYGON ((1165 748, 1164 749, 1163 749, 1162 7...",177.0
1,"POLYGON ((981 702, 980 703, 969 703, 968 704, ...",180.0
2,"POLYGON ((859 547, 858 548, 846 548, 845 549, ...",191.0
3,"POLYGON ((1086 635, 1085 636, 1081 636, 1080 6...",206.0
4,"POLYGON ((841 588, 840 589, 835 589, 834 590, ...",209.0
5,"POLYGON ((1058 529, 1057 530, 1051 530, 1050 5...",271.0
6,"POLYGON ((916 471, 915 472, 906 472, 905 473, ...",272.0
7,"POLYGON ((1168 616, 1167 617, 1164 617, 1163 6...",289.0
8,"POLYGON ((1711 568, 1710 569, 1709 569, 1708 5...",309.0
9,"POLYGON ((1556 459, 1555 460, 1550 460, 1549 4...",317.0


In [32]:
r_

,datakey,animalid,session,cell,x0,y0,fwhm_x,fwhm_y,theta,offset,...,minor_axis,anisotropy,theta_Mm_c,aniso_index,visual_area,experiment,x0_pix,y0_pix,fwhm_x_pix,fwhm_y_pix
197.0,20190508_JC083_fov1,JC083,20190508,197.0,53.957145,2.548797,27.746813,13.209007,0.060078,-0.005116,...,5.620854,0.354963,0.060078,-0.312338,Lm,average_rfs,1827,581,445.336347,212.004559
197.0,20190508_JC083_fov1,JC083,20190508,197.0,53.957145,2.548797,27.746813,13.209007,0.060078,-0.005116,...,5.620854,0.354963,0.060078,-0.312338,V1,average_rfs,1827,581,445.336347,212.004559


In [92]:
stim_polys = su.get_stimulus_polys(dk, experiment) 

In [93]:
o_=[]
for ri, rf_poly in rf_polys[['cell', 'poly']].values:
    res = su.cell_overlap_with_stimuli(ri, rf_poly, stim_polys)
    o_.append(res)
           

In [32]:
for si, spoly in enumerate(stim_polys['poly'].values):
    si

In [38]:
rfutils.calculate_overlap(rf_poly, spoly.buffer[0])

TypeError: 'method' object is not subscriptable

In [44]:
rf_poly + spoly

TypeError: unsupported operand type(s) for +: 'Polygon' and 'Polygon'

In [45]:
 rf_poly.intersection(spoly)

TopologyException: Input geom 0 is invalid: Self-intersection at or near point 1917 543 at 1917 543


TopologicalError: The operation 'GEOSIntersection_r' could not be performed. Likely cause is invalidity of the geometry <shapely.geometry.polygon.Polygon object at 0x2b6bf9a23ed0>

In [52]:
from shapely.validation import make_valid

ImportError: cannot import name 'make_valid' from 'shapely.validation' (/n/coxfs01/2p-pipeline/envs/rat2p/lib/python3.7/site-packages/shapely/validation.py)

In [51]:
validation.make_valid


AttributeError: module 'shapely.validation' has no attribute 'make_valid'

In [ ]:
rf_lim = 'percentile'
# cells_lim = dec.get_cells_with_matched_rfs(cells0, sdata, rf_lim='percentile')
rfdf = dec.get_rfdf(cells0, sdata, do_spherical_correction=False)
cells_RF = dec.get_cells_with_rfs(cells0, rfdf)
cells_lim, sz_limits = dec.limit_cells_by_rf(cells_RF, rf_lim=rf_lim)

sz_lower, sz_upper = sz_limits.loc['lower'].max(), lims.loc['upper'].min()
sz_limits

In [ ]:
print(cells_RF[['visual_area', 'datakey', 'cell']].drop_duplicates()\
      ['visual_area'].value_counts().to_markdown())

In [ ]:
%matplotlib notebook
fig, ax = pl.subplots(figsize=(5,4))
sns.stripplot(x='visual_area', y='std_avg', hue='visual_area', 
             data=cells_RF, ax=ax, order=visual_areas, 
             palette=area_colors, s=2, zorder=0)
sns.pointplot(x='visual_area', y='std_avg',
             data=cells_RF, ax=ax, order=visual_areas, join=False, ci='sd',
             estimator=np.median, color='k', markers='_', scale=1, zorder=1000,
             errwidth=1)

# sz_upper_lim=16.6
# sz_lower_lim=6.9
ax.axhline(y=sz_lower, color='k', ls=':', lw=0.5)
ax.axhline(y=sz_upper, color='k', ls=':', lw=0.5)
ax.legend_.remove()

fig.text(0.01, 0.95, 'Distribution of avg RF size, with median (%s)' % rf_lim)

pl.savefig(os.path.join(dst_dir, 'RF_sizes_%s.svg' % rf_lim))


In [ ]:
# Compare overlaps

In [ ]:
m1 = 'M%i' % 0
m2 = 'M%i' % 106
#images, impaths = su.load_stimuli(root='/n/home00/juliana.rhee', return_paths=True)
#print("im shape:", images['M0'].shape)
images = su.load_stimuli()
f, axn = pl.subplots(1,2, figsize=(5,2))
axn[0].imshow(images[m1]); axn[0].set_title(m1);
axn[1].imshow(images[m2]); axn[1].set_title(m2);


In [ ]:
## Example fov
dk = '20190602_JC091_fov1'
va='Li'
experiment='blobs'

In [ ]:
rid=15
rf_screen = su.rf_to_screen(rid, curr_rfs, resolution=[1920, 1080])
%matplotlib notebook
pl.figure()
pl.imshow(rf_screen)

In [ ]:
stim_polys, stim_screen = su.get_stimulus_polys

In [ ]:
rf = su.image_to_poly(rf_screen)
blob = su.image_to_poly(stim_screen.astype(np.uint8))
odf = rfutils.calculate_overlap(rf, blob, r1='rf', r2='blob')